## Get Posts

In [6]:
from vk_api import VkApi
from math import ceil
from tqdm import tqdm, trange

with open('token.txt', 'r') as tokens:
  token = tokens.readline()
  vk = VkApi(token = token)

pub_name = 'poiskmemow'
max_items = 100

In [7]:
count = vk.method("wall.get", {'domain': pub_name, 'count': 1})['count']
api_calls_count = ceil(count / max_items)
print(count, api_calls_count)

3435 35


In [8]:
posts_data = []
for i in trange(api_calls_count):
  request_data = {'domain': pub_name, 'count': max_items, 'offset': i * max_items}
  posts_data += vk.method("wall.get", request_data)['items']

100%|██████████| 35/35 [00:37<00:00,  1.08s/it]


In [9]:
import re

def format_text(text):
  url_re = r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)'
  src_url = f'([Ии]сточник.*?)?( {url_re})+'
  tags_re = r'#.+?@\S+'
  # clear text
  text = text.replace('\n', ' ').replace('\r', ' ') # remove newlines
  text = re.sub(src_url, '', text) # remove source links
  text = re.sub(url_re, '', text) # remove urls
  text = re.sub(tags_re, '', text) # remove hashtags
  text = text.replace('#', '') # convert hashtags to words
  text = ' '.join(text.split()).strip() # remove double spaces
  return text

In [30]:
class Post:
  def __init__(self, json):
    self.json, self.id = json, json['id']
    self.text, self.photos = self._parse_post(json)

  def _parse_post(self, json):
    text = format_text(json['text'])
    photos = []
    if 'attachments' in json.keys():
      for attachment in json['attachments']:
        if attachment['type'] == 'photo':
          max_src = max(attachment['photo']['sizes'], key=lambda x: x['width'] * x['height'])['url']
          photos.append(max_src)
    return text, photos
  
  def __str__(self):
    return f'Post(id={self.id}, text={self.text}, photos={self.photos})'

In [11]:
posts = [Post(post) for post in posts_data]
print("all posts:", len(posts))

posts = [post for post in posts if post.photos and post.text]
print("without images:", len(posts))

all posts: 3435
without images: 3404


In [12]:
with open('only_text.txt', 'w', encoding="utf-8") as f:
  for post in posts:
    f.write(f'{post.text}\n')

In [ ]:
from random import choice
print(choice(posts))

## Fuzzy Search

In [75]:
from fuzzywuzzy import process, utils, fuzz
from pprint import pprint

search_q = 'делайте хорошее'
text_list = [post.text for post in posts]
results = process.extract(search_q, text_list, limit=3, scorer=fuzz.partial_ratio)

pprint(results) # returns list of tuples (text, score)

[('Сдраствуйте здравствуйте делайте делойте хорошее а плохое не делойте '
  'досведания до свидания красный фон кошка просит кот светлый вылезает из '
  'сердечка в рамке в форме сердца с белым кружевом лапка лапа вперёд '
  'зацепился добрая картинка с подписью открытка',
  93),
 ('просто супер отлично но переделайте вахрушева т.н. печать штамп шамова л.а. '
  'выставочный синий индивидуальный предприниматель российская федерация город '
  'г киров область тетрадь в клетку клеточку 10.07 10 десять десятое июля июль '
  '07 7',
  80),
 ('пожелание хорошего вечера ночи чай приятного печенье чаепитие чайник ужин '
  'вечер хорошо ложка в чашке лимон черный чай скатерть шрифт лобстер',
  73)]


In [62]:
images = {post.id: post.photos for post in posts}
keywords = {post.id: post.text for post in posts}

In [ ]:
from fuzzywuzzy import process, utils, fuzz
from pprint import pprint

search_q = 'котик'
results = process.extract(search_q, keywords, limit=3, scorer=fuzz.partial_ratio)
results = [
  {
    "text": result[0],
    "score": result[1],
    "id": result[2],
    "images": images[result[2]]
  }
  for result in results
]

pprint(results)

In [72]:
from IPython.display import Image, display

for result in results:
  print(result['text'])
  for url in result['images']:
    display(Image(url=url, width=350, unconfined=True))

Невдупленыш котенок котик кот кошка кошечка черный черная смотрит глазет растерянно растерянный не понимает не вдупляет блесящие глаза глазки человек белая футболка ковёр шрифт лобстер белая подпись


ну вот мы и встретились питса пицца пица встреча сюрприз кот кошка котик смотрит на пиццу нашёл пиццу пицца на тарелке тарелка с пиццей тянется к пицце пицценька не переживунькай всё будет хорошо не переживай не волнуйся так не тревожься не грусти пицца пица питса кот котик кошка гладит пиццу тянется к пицце коробка с пиццей ахаха глупая питса от меня не спрячешся пицца пица не спрячешься кот котик кошка смотрит на пиццу подглядывает коробка с пиццей дырка в коробке окошко отверстие прятки питса выходи я прешол с миром пицца пица пришёл с миром не с миром кот котик кошка смотрит на пиццу коробка с пиццей дырка в коробке окошко отверстие проход туннель тук тук готова готов сцапать съесть скушать успокаивает вводит в ложное чувство спокойствия доверия вкусная сырная пепперони овощная с оливками и перцем европейская с колбаской


Кот котик плачет мем не смешно картинка пикча picture боль больно тема больная сука довели плачет слезы из глаз слезы сука текст мотеша математика завтра неет слезы боль депрессия кот плакает студент школьник студенческие слезы домашка кот не надо белый не хочет на математику наверное домашку не сделал мотеша завтра сука два 2 что((((
